<a href="https://colab.research.google.com/github/GurashishJuneja/MMAI-891-NLP-Project/blob/main/Irene_Copy_using_GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1.Initial import of packages and mount**


In [1]:
from google.colab import drive
drive.mount('/content/MyDrive')

Mounted at /content/MyDrive


In [2]:
!pip install transformers
!pip install sentencepiece

import pandas as pd
import numpy as np
import torch
import transformers
from sklearn.model_selection import train_test_split
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, Trainer, TrainingArguments
# Load the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

import numpy as np
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense, Input, Dropout

import tensorflow as tf
from sklearn.preprocessing import LabelEncoder



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 84.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.1 MB/s eta 0:00:00


In [3]:
# mount Google Drive to Google Colab notebook
from google.colab import drive
drive.mount('/content/NewDrive')

Mounted at /content/NewDrive


In [4]:
train_df = pd.read_csv("/content/NewDrive/MyDrive/NLP/Train.csv")
test_df = pd.read_csv("/content/NewDrive/MyDrive/NLP/Test.csv")

In [5]:
# define valid label values
valid_labels = [-1, 0, 1]

# filter out rows with invalid label values
train_df = train_df[train_df['label'].isin(valid_labels)]

# reset the index after filtering
train_df = train_df.reset_index(drop=True)

test_df = test_df.dropna(subset=['safe_text'])

In [6]:
train_df

,tweet_id,safe_text,label,agreement
0,CL1KWCMY,Me &amp; The Big Homie meanboy3000 #MEANBOY #M...,0.0,1.000000
1,E3303EME,I'm 100% thinking of devoting my career to pro...,1.0,1.000000
2,M4IVFSMS,"#whatcausesautism VACCINES, DO NOT VACCINATE Y...",-1.0,1.000000
3,1DR6ROZ4,I mean if they immunize my kid with something ...,-1.0,1.000000
4,J77ENIIE,Thanks to <user> Catch me performing at La Nui...,0.0,1.000000
...,...,...,...,...
9994,IU0TIJDI,Living in a time where the sperm I used to was...,1.0,1.000000
9995,WKKPCJY6,<user> <user> In spite of all measles outbrea...,1.0,0.666667
9996,ST3A265H,Interesting trends in child immunization in Ok...,0.0,1.000000
9997,6Z27IJGD,CDC Says Measles Are At Highest Levels In Deca...,0.0,1.000000


In [7]:
train_df['label'] = train_df['label'].map({-1: 0, 0: 1, 1: 2})  # map values to 0, 1, and 2

In [8]:
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=24)

In [9]:
def clean_data(df):
 
  df=df.dropna()
    
  # Remove duplicate entries
  clean_data = df.drop_duplicates(subset=['safe_text'])
 
  return clean_data

train = clean_data(train_df)
val = clean_data(val_df)

In [10]:
# Load the pre-trained tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# Load the GPT model for sequence classification
model = GPT2ForSequenceClassification.from_pretrained('gpt2')

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

train_texts = train['safe_text'].tolist()
train_labels = train['label'].tolist()

val_texts = val['safe_text'].tolist()
val_labels = val['label'].tolist()

test_texts = test_df['safe_text'].tolist()

onehot_encoder = OneHotEncoder(sparse=False)
train_labels = np.array(train_labels)
val_labels = np.array(val_labels)
onehot_train = onehot_encoder.fit_transform(train_labels.reshape(-1, 1))
onehot_val = onehot_encoder.fit_transform(val_labels.reshape(-1, 1))


/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [13]:
def encode_text(text):
    # TODO: encode text using dbert_tokenizer
    # DO NOT CHANGE THE INPUTS OR OUTPUTS TO THIS FUNCTION
    model_inputs_and_masks = tokenizer(text,padding='max_length', truncation=True, return_attention_mask=True,add_special_tokens=True,return_tensors="tf",max_length = 128)

    input_ids = model_inputs_and_masks['input_ids']
    attention_mask = model_inputs_and_masks['attention_mask']
    return input_ids, attention_mask


train_input, train_mask = encode_text(train_texts)
val_input, val_mask = encode_text(val_texts)
test_input, test_mask = encode_text(test_texts)

train_model_inputs_and_masks = {
    'inputs' : train_input,
    'masks' : train_mask
}

val_model_inputs_and_masks = {
    'inputs' : val_input,
    'masks' : val_mask
}

test_model_inputs_and_masks = {
    'inputs' : test_input,
    'masks' : test_mask
}

In [14]:
train_input

<tf.Tensor: shape=(7757, 128), dtype=int32, numpy=
array([[   40,  1101,   262, ..., 50257, 50257, 50257],
       [    1,    40,  2314, ..., 50257, 50257, 50257],
       [ 5211,   314,  3285, ..., 50257, 50257, 50257],
       ...,
       [43920,  5156,   327, ..., 50257, 50257, 50257],
       [25147,   340,   338, ..., 50257, 50257, 50257],
       [48374, 16989, 27496, ..., 50257, 50257, 50257]], dtype=int32)>

In [15]:
train_inputs = np.array(train_model_inputs_and_masks['inputs'])
train_masks = np.array(train_model_inputs_and_masks['masks'])
val_inputs = np.array(val_model_inputs_and_masks['inputs'])
val_masks = np.array(val_model_inputs_and_masks['masks'])
test_inputs = np.array(test_model_inputs_and_masks['inputs'])
test_masks = np.array(test_model_inputs_and_masks['masks'])

In [16]:
onehot_train

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.]])

In [45]:
import tensorflow as tf
from transformers import TFGPT2Model

def build_model(base_model, trainable=False):
    # Refer to https://keras.io/api/layers/core_layers/input/
    max_seq_len = 128
    inputs = tf.keras.Input(shape=(max_seq_len,), dtype=tf.int32, name="input_ids")
    masks = tf.keras.Input(shape=(max_seq_len,), dtype=tf.int32, name="attention_mask")

    base_model.trainable = trainable

    gpt2_output = base_model(inputs, attention_mask=masks)
    gpt2_last_hidden_state = gpt2_output.last_hidden_state

    my_output = gpt2_last_hidden_state[:,0]
    probs = tf.keras.layers.Dense(512, activation='relu')(my_output)
    probs1 = tf.keras.layers.Dense(256, activation='relu')(probs)
    probs2 = tf.keras.layers.Dense(128, activation='relu')(probs1)
    probs3 = tf.keras.layers.Dense(3, activation='softmax')(probs2)

    model = tf.keras.Model(inputs=[inputs, masks], outputs=probs3)
    model.summary()
    return model
    model_outputs = model([inputs, masks])

model = build_model(TFGPT2Model.from_pretrained('gpt2'), trainable=True)






All model checkpoint layers were used when initializing TFGPT2Model.

All the layers of TFGPT2Model were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2Model for predictions without further training.


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 128)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 tfgpt2_model_1 (TFGPT2Model)   TFBaseModelOutputWi  124439808   ['input_ids[0][0]',              
                                thPastAndCrossAtten               'attention_mask[0][0]']         
                                tions(last_hidden_s                                               
                                tate=(None, 128, 76                                         

In [52]:
def compile_model(model):

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC','accuracy'])
    
    return model

model = compile_model(model)

In [57]:
import torch
import torch.nn as nn

class model(nn.Module):
    def __init__(self):
        super(model, self).__init__()
        self.fc1 = nn.Linear(in_features=10, out_features=20)
        self.fc2 = nn.Linear(in_features=20, out_features=2)
    
    def forward(self, inputs):
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        return x
        
        
model = model()


In [77]:
batch_size = 32
learning_rate = 2e-5
num_epochs = 2

import torch
from torch.utils.data import TensorDataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')


optimizer = AdamW(model.parameters(), lr=learning_rate)

model.train()
for epoch in range(num_epochs):
    train_loss = 0
    for i, (input_ids, attention_masks, labels) in enumerate(train_dataloader):
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_masks, labels=labels)
        loss = outputs.loss
        train_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
    train_loss /= len(train_dataloader)
    print(f'Epoch {epoch+1} train loss: {train_loss}')










IndexError: ignored

In [27]:
logits = model(test_inputs, attention_mask=test_masks)[0]




TypeError: ignored

In [ ]:
logits.shape

In [ ]:
logits

In [ ]:
# Define the labels
labels = [-1, 0, 1]

# Get the index of the highest probability for each example
indices = np.sum(logits* np.array(labels),axis = 1)



In [ ]:
indices 

In [ ]:
output_df = pd.DataFrame({'ID': test_df['tweet_id'], 'target': indices})
output_df.to_csv('/content/NewDrive/MyDrive/NLP/predictions.csv', index=False)